# Installation & Maintenance





## First Install

### Trust the PSGallery Repository

In [ ]:
Set-PSRepository -name PSGallery -InstallationPolicy Trusted

In [ ]:
Install-Module -Name dbatools -Scope CurrentUser;

## Updating dbatools

In [ ]:
Update-Module -Name dbatools -Verbose;

# Basic Backup Information
* Where do backups go?
* Do we have backups?
* Do they get compressed?

In [1]:
$SqlInstance = "localhost\sql17";
Get-DbaDefaultPath -SqlInstance $SqlInstance;



ComputerName : FLEXO
InstanceName : SQL17
SqlInstance  : FLEXO\SQL17
Data         : C:\SQL\Data\SQL17
Log          : C:\SQL\Log\SQL17
Backup       : C:\SQL\Backup
ErrorLog     : C:\SQL\Data\MSSQL14.SQL17\MSSQL\Log





### Open an Explorer window to the backup location

In [ ]:
$SqlInstance = "localhost\sql17";
Get-DbaDefaultPath -SqlInstance  $SqlInstance | Select-Object -ExpandProperty Backup | Invoke-Item;

### Check & Set Backup Compression

In [ ]:
$SqlInstance = "localhost\sql17";
Get-DbaSpConfigure -SqlInstance $SqlInstance -Name DefaultBackupCompression;

In [ ]:
$SqlInstance = "localhost\sql17";
Set-DbaSpConfigure -SqlInstance $SqlInstance -Name DefaultBackupCompression -Value 1;

## Let's Work With Backups!

When did we last back up our databases?

In [4]:
Get-DbaLastBackup -SqlInstance $SqlInstance | Format-Table -AutoSize;

ComputerName InstanceName SqlInstance Database        LastFullBackup          LastDiffBackup LastLogBackup          
------------ ------------ ----------- --------        --------------          -------------- -------------          
FLEXO        SQL17        FLEXO\SQL17 CacheDB         2020-11-23 21:00:09.000                2020-11-24 13:30:03.000
FLEXO        SQL17        FLEXO\SQL17 cachedb2        2020-11-23 21:00:16.000                2020-11-24 13:30:03.000
FLEXO        SQL17        FLEXO\SQL17 cachedb-backup  2020-11-23 21:00:17.000                2020-11-24 13:30:04.000
FLEXO        SQL17        FLEXO\SQL17 DBA             2020-11-23 21:00:17.000                2020-11-24 13:30:04.000
FLEXO        SQL17        FLEXO\SQL17 dbaclone        2020-11-23 21:00:17.000                2020-11-24 13:30:04.000
FLEXO        SQL17        FLEXO\SQL17 DBAThings       2020-11-23 21:00:18.000                                       
FLEXO        SQL17        FLEXO\SQL17 Demo            2020-11-23

### Backup history for one database

In [ ]:
Get-DbaDbBackupHistory -SqlInstance $SqlInstance -Database DBA

### Recent Backups

In [ ]:
Get-DbaDbBackupHistory -SqlInstance $SqlInstance -IncludeCopyOnly -Since (Get-Date).AddDays(-7) -DeviceType Disk | Sort-Object -property Start;

Let's send that to Excel instead

In [14]:
Get-DbaDbBackupHistory -SqlInstance $SqlInstance -IncludeCopyOnly -Since (Get-Date).AddDays(-7) -DeviceType Disk | Sort-Object -property Start | Export-Excel -Path C:\users\andy\documents\BackupHistory.xlsx -AutoSize -FreezeTopRow -Show;

# Ad hoc Backups

In [ ]:
$Sql17 = "localhost\sql17";
Get-DbaDefaultPath -SqlInstance $Sql17;
$BackupParameters = @{
    SqlInstance = $Sql17;
    Type = "Full";
    Database = "imdb";
    CopyOnly = $true;
    CompressBackup = $true;

}
Backup-DbaDatabase @BackupParameters;

### Automated Backups

In [ ]:
$SqlInstance = "localhost\sql17";
Install-DbaMaintenanceSolution -SqlInstance $SqlInstance -Solution All -Database DBAThings -CleanupTime 97 -InstallJobs -ReplaceExisting -LogToTable;

$EveryFiveMinutes = New-DbaAgentSchedule -SqlInstance $SqlInstance -Schedule "Five Minutes" -FrequencyType Daily -FrequencyInterval 1 -FrequencySubdayType Minutes -FrequencySubdayInterval 5 -Force;
$EveryFifteenMinutes = New-DbaAgentSchedule -SqlInstance $SqlInstance -Schedule "Five Minutes" -FrequencyType Daily -FrequencyInterval 1 -FrequencySubdayType Minutes -FrequencySubdayInterval 15 -Force;

Backup-DbaDatabase -SqlInstance localhost\sql17 -Type Full